In this work, you will use the APIs of PyG and DGL to implement some basic functions.

You need to run the following commands to install the GNN libraries (Only CPU version).


The most popular GNN models can be written as follows:

$$
h_i^{(l+1)}=\sigma(b^{(l)}+\sum_{j\in\mathcal{N}(i)}e_{ij}h_j^{(l)}W^{(l)})
$$

where $h_i^{(l+1)}$ is the output feature, $\sigma$ is the activation function, $e_{ij}$ is the edge weight, $W^{(l)}$ is the learnable parameters, $b^{(l)
}$ is the bias.



In [5]:
import torch
import torch.nn as nn
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.utils import add_self_loops
class PyG_conv(MessagePassing):
  def __init__(self,in_channel,out_channel,):
    super(PyG_conv, self).__init__(aggr='add')
    self.in_channel = in_channel
    self.out_channel = out_channel
    self.W = nn.Parameter(torch.ones((in_channel, out_channel)))
    self.b = nn.Parameter(torch.ones(out_channel))

  def forward(self, x, edge_index, edge_weight):
    # Your code here
     x_p = self.propagate(edge_index, x=x, edge_weight=edge_weight)
    # End code here

  def message(x, edge_weight):
    # Your code here
    
    # End code here

SyntaxError: unexpected EOF while parsing (3132688240.py, line 21)

In [ ]:
import torch
import numpy as np
edge_index = torch.tensor([[0,1,1,2,2,4],[2,0,2,3,4,3]])
x = torch.ones((5, 8))
edge_weight = 2 * torch.ones(6)
conv = PyG_conv(8, 4)
output = conv(x, edge_index, edge_weight)
print(output)
assert np.allclose(output.detach().numpy(), [[17., 17., 17., 17.],
                      [ 1.,  1.,  1.,  1.],
                      [33., 33., 33., 33.],
                      [33., 33., 33., 33.],
                      [17., 17., 17., 17.]])

In [7]:
import torch
import dgl
import torch.nn as nn

In [15]:
#create the class about the conv

class DGL_conv(nn.Module):
    def __init__(self, input_channels, output_channels):
        super(DGL_conv, self).__init__()
        self.input_channel = input_channels
        self.output_channel = output_channels
        self.W = nn.Parameter(torch.ones((input_channels,output_channels)))
        self.b = nn.Parameter(torch.ones(output_channels))
    def forward (self, g, h, edge_weight):
        with g.local_scope():# local_scope 函数的目的是在这个区域进行信息传递，聚合和更新，但是并不修改原图的节点或边的特征
            g.edata['w'] = edge_weight
            h = torch.matmul(h, self.W)
            # h = torch.matmul(h,g.edges['w'])
            g.ndata['h'] = h
            g.update_all(dgl.function.u_mul_e('h','w', 'm'), dgl.function.sum('m', 'h_sum'))
            h_sum = g.ndata['h_sum']
            h_message = h_sum + self.b
            return h_message

In [16]:
import numpy as np
src = torch.tensor([0, 1, 1, 2, 2, 4])
dst = torch.tensor([2, 0, 2, 3, 4, 3])
h = torch.ones((5, 8))
g = dgl.graph((src, dst))
edge_weight = 2 * torch.ones(6)
# g.edata['w'] = edge_weight
conv = DGL_conv(8, 4)
output = conv(g, h, edge_weight)       
print(output)
print(np.allclose(output.detach().numpy(), [[17., 17., 17., 17.],
                      [ 1.,  1.,  1.,  1.],
                      [33., 33., 33., 33.],
                      [33., 33., 33., 33.],
                      [17., 17., 17., 17.]]))

tensor([[17., 17., 17., 17.],
        [ 1.,  1.,  1.,  1.],
        [33., 33., 33., 33.],
        [33., 33., 33., 33.],
        [17., 17., 17., 17.]], grad_fn=<AddBackward0>)
True


In [27]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops

class PyG_conv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(PyG_conv, self).__init__(aggr='add')  # 聚合函数为加法
        self.input_channel = in_channels
        self.output_channel = out_channels
        self.W = torch.nn.Parameter(torch.Tensor(in_channels, out_channels))
        self.bias = torch.nn.Parameter(torch.Tensor(out_channels))
        torch.nn.init.xavier_uniform_(self.W)  # 使用Xavier初始化
        torch.nn.init.zeros_(self.bias)

    def forward(self, x, edge_index,edge_weight):
        # 可选：为邻接矩阵添加自环
        # edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
        
        print("edge_index:", edge_index.shape)
        print("x before linear:", x.shape)
        
        # 线性变换
        x = torch.matmul(x, self.W)
        
        print("x after linear:", x.shape)
        
        # 开始消息传递
        out = self.propagate(edge_index, x=x,edge_weight=edge_weight)
        
        # 添加偏置
        out += self.bias
        return out

    # 重写message函数
    def message(self, x_j, edge_weight):
        print("x_j:", x_j.shape)
        if edge_weight is None:
            return x_j
        else:
            print("edge_weight:", edge_weight.shape)
            return edge_weight.view(-1, 1) * x_j

    # 如有需要，可以重写update函数（可选）
    def update(self, aggr_out):
        return aggr_out


In [28]:
# 使用示例
from torch_geometric.data import Data
import numpy as np
edge_index = torch.tensor([[0, 1, 1, 2, 2, 4], [2, 0, 2, 3, 4, 3]])
x = torch.ones((5, 8))
edge_weight = 2 * torch.ones(6)
data = Data(x=x, edge_index=edge_index,edge_weight = edge_weight)
conv = PyG_conv(8, 4)
output = conv(data.x, data.edge_index, data.edge_weight)
print(output)
assert np.allclose(output.detach().numpy(), [[17., 17., 17., 17.],
                      [ 1.,  1.,  1.,  1.],
                      [33., 33., 33., 33.],
                      [33., 33., 33., 33.],
                      [17., 17., 17., 17.]])

edge_index: torch.Size([2, 11])
x before linear: torch.Size([5, 8])
x after linear: torch.Size([5, 4])
x_j: torch.Size([11, 4])
edge_weight: torch.Size([6])


RuntimeError: The size of tensor a (6) must match the size of tensor b (11) at non-singleton dimension 0